The purpose of this Notebook is to illustrate a simple workflow for testing and evaluating LLM applications. We will also learn how to evalaute a RAG system using ragas. The evaluation framework uses the [ragas](https://docs.ragas.io/en/latest/) libary.

**NOTE:** *The content in this notebook are mostly from the Ragas documentation and tutorial guide.*

### **Installation**
To get started, install Ragas and chat model(s) using pip with the following command:

In [ ]:
%pip --quiet install ragas
%pip install --quiet langchain-google-genai
%pip install --quiet sacrebleu
%pip install --quiet --upgrade langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.7/190.7 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.

### **1. Evaluate Direct LLM**

In the example below:



*   A test sample containing user_input, response (the output from the LLM), and reference (the expected output from the LLM) as data points to evaluate the summary.
*   The **BleuScore** metric is used to evaluate the quality of response by comparing it with reference. It measures the similarity between the response and the reference based on n-gram precision and brevity penalty. BLEU score ranges from 0 to 1, where 1 indicates a perfect match between the response and the reference.


---


**Limitations with BlueScore**



*   Time-Consuming Preparation: Evaluating the application requires preparing the expected output (reference) for each input, which can be both time-consuming and challenging.
*   Inaccurate Scoring: Even though the response and reference are similar, the output score was low.

In [ ]:
import getpass
import os
import bs4
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
# os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
# os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
user_input = "What is HPC?"   # type in your input, e.g., a question
reference = "HPC (High-Performance Computing) is a technology that utilizes clusters of powerful processors working in parallel to process massive, multidimensional data sets and solve complex problems at extremely high speeds. HPC systems typically run more than one million times faster than the fastest commodity desktop, laptop, or server systems. It employs massively parallel computing, using tens of thousands to millions of processors or processor cores. HPC clusters consist of multiple high-speed computer servers networked with a centralized scheduler to manage the parallel computing workload. These computers, called nodes, often use high-performance multi-core CPUs or GPUs. All other computing resources within an HPC cluster, such as networking, memory, storage, and file systems, are also high-speed, high-throughput, and low-latency components."    # this is the ground-truth answer

# Create an LLM model e.g., OpenAI GPT-4o or Google Gemini-2.5
llm = ChatOpenAI(model="gpt-4o")
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

result = llm.invoke(user_input)
response = result.content

print(f"User Input: {user_input}")
print(f"Reference: {reference}")
print(f"LLM Response: {response}")

User Input: What is HPC?
Reference: HPC (High-Performance Computing) is a technology that utilizes clusters of powerful processors working in parallel to process massive, multidimensional data sets and solve complex problems at extremely high speeds. HPC systems typically run more than one million times faster than the fastest commodity desktop, laptop, or server systems. It employs massively parallel computing, using tens of thousands to millions of processors or processor cores. HPC clusters consist of multiple high-speed computer servers networked with a centralized scheduler to manage the parallel computing workload. These computers, called nodes, often use high-performance multi-core CPUs or GPUs. All other computing resources within an HPC cluster, such as networking, memory, storage, and file systems, are also high-speed, high-throughput, and low-latency components.
LLM Response: HPC stands for High-Performance Computing. It refers to the practice of using supercomputers and par

In [ ]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": user_input,
    "response": response,
    "reference": reference
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.04773548444510098

### **2. Evaluate RAG-assisted LLM**

In [ ]:
import numpy as np

class RAG:
    def __init__(self, llm_model, embedding_model):
        # self.llm = ChatGoogleGenerativeAI(model=llm_model)
        # self.embeddings = GoogleGenerativeAIEmbeddings(model=embedding_model)
        self.llm = ChatOpenAI(model=llm_model)
        self.embeddings = OpenAIEmbeddings(model=embedding_model)
        self.doc_embeddings = None
        self.docs = None

    def load_documents(self, documents):
        """Load documents and compute their embeddings."""
        self.docs = documents
        self.doc_embeddings = self.embeddings.embed_documents(documents)

    def get_most_relevant_docs(self, query):
        """Find the most relevant document for a given query."""
        if not self.docs or not self.doc_embeddings:
            raise ValueError("Documents and their embeddings are not loaded.")

        query_embedding = self.embeddings.embed_query(query)
        similarities = [
            np.dot(query_embedding, doc_emb)
            / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_emb))
            for doc_emb in self.doc_embeddings
        ]
        most_relevant_doc_index = np.argmax(similarities)
        return [self.docs[most_relevant_doc_index]]

    def generate_answer(self, query, relevant_doc):
        """Generate an answer for a given query based on the most relevant document."""
        prompt = f"question: {query}\n\nDocuments: {relevant_doc}"
        messages = [
            ("system", "You are a helpful assistant that answers questions based on given documents only."),
            ("human", prompt),
        ]
        ai_msg = self.llm.invoke(messages)
        return ai_msg.content

In [ ]:
# create the list of web contents to be used for RAG pipeline
web_paths = ["https://www.ibm.com/think/topics/hpc",
             "https://www.geeksforgeeks.org/computer-organization-architecture/high-performance-computing/"]

# Load documents using LangChain WebBaseLoader
documents = []

# Read the text content from the file
for url in web_paths:
    try:
        # Load and chunk contents of the blog
        loader = WebBaseLoader(url)
        data = loader.load()
        for doc in data:
            # Add metadata to each document indicating its source
            doc.metadata = {"source": url}
            documents.append(doc)
        print(f"[✓] Loaded {url}")
    except Exception as e:
        print(f"[✗] Failed to load {url}: {e}")

for doc in documents:
    doc.page_content = " ".join(doc.page_content.split())  # remove white space

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
chunked_docs = [doc.page_content for doc in docs]

[✓] Loaded https://www.ibm.com/think/topics/hpc
[✓] Loaded https://www.geeksforgeeks.org/computer-organization-architecture/high-performance-computing/


In [ ]:
# llm_model = "gemini-2.5-flash"
# embedding_model = "models/gemini-embedding-001"
llm_model = "gpt-4o"
embedding_model = "text-embedding-3-small"

# Initialize RAG instance
rag = RAG(llm_model, embedding_model)

# Load documents
rag.load_documents(chunked_docs)

In [ ]:
# Query and retrieve the most relevant document
query = "How do I submit a job to a supercomputer?"
relevant_doc = rag.get_most_relevant_docs(query)

# Generate an answer
answer = rag.generate_answer(query, relevant_doc)

print(f"Query: {query}")
print(f"Relevant Document: {relevant_doc}")
print(f"Answer: {answer}")

Query: How do I submit a job to a supercomputer?
Relevant Document: ['supercomputers are much faster and can run billions of floating-point operations in one second. Supercomputers are still with us; the fastest supercomputer is the US-based Frontier, with a processing speed of 1.206 exaflops or quintillion floating point operations per second (flops).1 But today, more organizations are running HPC services on clusters of high-speed computer servers hosted on premises or in the cloud. HPC workloads uncover new insights that advance human knowledge and create significant competitive advantages. For example, HPC sequences DNA and automates stock trading. It runs artificial intelligence (AI) algorithms and simulations—like those enabling self-driving automobiles—that analyze terabytes of data streaming from IoT sensors, radar and GPS systems in real-time to make split-second decisions. Industry newsletter The latest tech news, backed by expert insights Stay up to date on the most importan

Collect and/or Load your Evaluation Data

In [ ]:
sample_batch = [
  {
    "question": "What is High-Performance Computing (HPC)?",
    "answer": "High-Performance Computing (HPC) refers to the use of parallel processing by networking multiple computers (often clustered) to achieve performance far beyond that of typical desktops. These systems enable efficient and fast execution of advanced applications, particularly those reaching teraflop-scale computing — a trillion floating-point operations per second."
  },
  {
    "question": "Why is HPC critically important?",
    "answer": "HPC is important because it pushes scientific discovery, supports complex modeling (e.g., AI, climate modeling, nuclear physics), manages massive datasets in business analytics and IoT, and accelerates areas like drug discovery, 3D imaging, and genome sequencing."
  },
  {
    "question": "What are the primary needs that drive the adoption of HPC?",
    "answer": "Organizations adopt HPC to reduce runtime for lengthy computations, meet tight deadlines by achieving high operations per second, and use parallel CPU and GPU resources efficiently through fast inter-node networks."
  },
  {
    "question": "How does an HPC system operate?",
    "answer": "An HPC system operates with a scheduler that distributes jobs to compute clusters, where each node runs parts of the workload in parallel. Data storage provides fast I/O for computations. Synchronization across nodes, networking, and storage is critical to overall efficiency."
  },
  {
    "question": "What are the key challenges associated with HPC?",
    "answer": "Challenges include high cost (hardware, software, energy, personnel), scalability issues, massive data management needs, complexity of parallel programming, software compatibility constraints, and significant power and cooling requirements."
  }
]

In [ ]:
dataset = []

for pair in sample_batch:
  query = pair["question"]
  reference = pair["answer"]

  relevant_docs = rag.get_most_relevant_docs(query)
  response = rag.generate_answer(query, relevant_docs)
  dataset.append(
      {
          "user_input":query,
          "retrieved_contexts":relevant_docs,
          "response":response,
          "reference":reference
      }
  )

Now, we can evaluate our RAG system on the collected dataset using a set of commonly used RAG evaluation metrics. You may choose any model as evaluator LLM for evaluation.

### Generation-focused metrics:

1. **faithfulness**:
Measures how factual the generated answer is with respect to the provided contexts.
It evaluates if the answer contains information that isn't present in or contradicts the retrieved contexts.

2. **answer_relevancy**:
Assesses how well the generated answer addresses the user's query, regardless of factuality.
It measures if the answer is on-topic and responsive to what was asked.

### Retieval-focused metrics:

1. **context_relevance**:
Evaluates how relevant the retrieved contexts are to the user's query.
This metric helps determine if the retrieval component is returning useful information.

2. **context_recall**:
Measures how completely the retrieved contexts cover the information needed to answer the query.
When ground truth is available, it assesses if all necessary information was retrieved.

3. **context_precision**:
Evaluates how focused and precise the retrieved contexts are.
It measures whether the contexts contain mostly relevant information without excessive irrelevant content.

In [ ]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# Create an LLM model e.g., OpenAI GPT-4o or Google Gemini-2.5
llm = ChatOpenAI(model="gpt-4o")
# llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

evaluator_llm = LangchainLLMWrapper(llm)
result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],llm=evaluator_llm)
print(result)

Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
print(result)